[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rslab-ntua/MSc_GBDA/blob/master/2022/Lab1a.ipynb)

In [7]:
# Download data, unzip
!gdown https://drive.google.com/uc?id=1XxBBah4J3wmSAMFq8lBFc06vGWFiy1TZ
!unzip GBDA2020_ML1.zip

In [ ]:
# Install dependencies
!pip install rasterio

In [ ]:
import rasterio
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

DATA_ROOT = "partA/"

# Read data
with rasterio.open(os.path.join(DATA_ROOT, "green.tif")) as dataset:
    green = dataset.read(1).astype(np.float32)
with rasterio.open(os.path.join(DATA_ROOT, "nir.tif")) as dataset:
    nir = dataset.read(1).astype(np.float32)
with rasterio.open(os.path.join(DATA_ROOT, "gt.tif")) as dataset:
    gt = dataset.read(1).astype(int)

print(f"Groundtruth \tmax: {gt.min()}, min: {gt.max()}, dtype: {gt.dtype}, classes: {np.unique(gt)}")
print(f"Green channel \tmax: {gt.min()}, min: {gt.max()}, dtype: {gt.dtype}")
print(f"NIR channel \tmax: {gt.min()}, min: {gt.max()}, dtype: {gt.dtype}")

plt.figure(dpi=300)
plt.subplot(1, 3, 1)
plt.title("GREEN")
plt.imshow(green, cmap="gray")
plt.axis('off')
plt.subplot(1, 3, 2)
plt.title("NIR")
plt.imshow(nir, cmap="gray")
plt.axis('off')
plt.subplot(1, 3, 3)
plt.title("GroundTruth")
plt.imshow(gt, cmap="gray")
plt.axis('off')
plt.show()


In [ ]:
# Classic Remote Sensing approach
# Compute NDWI
ndwi = (green-nir)/(green+nir)

# Threshold
ndwi_h = ndwi >= 0.3

print(f"NDWI \tmax: {ndwi.min()}, min: {ndwi.max()}, mean: {ndwi.mean()} dtype: {ndwi.dtype}")


plt.figure(dpi=300)
plt.subplot(1, 2, 1)
plt.title("NDWI")
plt.imshow(ndwi, cmap="gray")
plt.axis('off')
plt.subplot(1, 2, 2)
plt.title("Thresholded")
plt.imshow(ndwi_h, cmap="gray")
plt.axis('off')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
# Generate dataset, splits

X = np.stack([green, nir], axis=-1).reshape(-1, 2)
y = (gt / gt.max()).reshape(-1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

print(f"X: {X.shape}, y: {y.shape}")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y: {y_test.shape}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Classic ML methods

clf = RandomForestClassifier(max_depth=2)
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)

tp = (predictions[predictions==1] == y_test[predictions==1]).sum()
fp = (predictions[predictions==1] != y_test[predictions==1]).sum()
fn = (predictions[predictions==0] != y_test[predictions==0]).sum()
tn = (predictions[predictions==0] == y_test[predictions==0]).sum()
print(tp, fp, fn, tn)

acc = (tp+tn)/(tp+fp+fn+tn)
pre = tp/(tp+fp)
rec = tp/(tp+fn)
f1 = 2*(pre*rec) / (pre+rec)



print("Accuracy: ", acc, accuracy_score(y_test, predictions))
print("Precision: ", pre, precision_score(y_test, predictions))
print("Recall: ", rec, recall_score(y_test, predictions))
print("F1-score: ", f1, f1_score(y_test, predictions))

In [ ]:
from sklearn.linear_model import Perceptron

clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)
print("Accuracy: ", accuracy_score(y_test, predictions))

print(clf.coef_, clf.coef_.shape)
print(clf.intercept_, clf.intercept_.shape)

In [ ]:
# Perceptron with Pytorch
import torch
from torch import nn

class Perceptron(nn.Module):
    def __init__(self, in_features: int):
        super().__init__()
        self.W = torch.normal(mean=in_features, std=in_features, size=(1,2), requires_grad=True).type(torch.float32)
        self.b = torch.normal(mean=0.0, std=1.0, size= (), requires_grad=True).type(torch.float32)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        print(x.type(), self.W.type())
        return torch.sigmoid(x @ self.W.T + self.b)


pp = Perceptron(2)
# Set manually weights and biases
pp.W = torch.from_numpy(clf.coef_).type(torch.float32)
pp.b = torch.from_numpy(clf.intercept_).type(torch.float32)

predictions = pp(torch.from_numpy(X_test)) > 0.5
print(predictions.size())
print("Accuracy: ", accuracy_score(y_test, predictions.flatten()))


pp2 = nn.Sequential(nn.Linear(2,1), nn.Sigmoid())
for p in pp2.parameters():
    if p.data.size() == (1, 2):
        p.data = torch.from_numpy(clf.coef_).type(torch.float32)
    else:
        p.data = torch.from_numpy(clf.intercept_).type(torch.float32)

predictions = pp2(torch.from_numpy(X_test)) > 0.5
print(predictions.size())
print("Accuracy: ", accuracy_score(y_test, predictions.flatten()))